In [2]:
import pandas as pd
from pandas import read_csv
import glob
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load all the blocks and concatenate them into one database
path = '/Users/mariabelenalberti/OneDrive - Universidad Torcuato Di Tella/Tesis/smart-meters-in-london/halfhourly_dataset' # use your path
all_files = glob.glob(path + "/*.csv")

lista = []
i = 0
for filename in all_files:
    i = i + 1
    print(i) # should go up to 112
    df = pd.read_csv(filename, index_col=None, header=0)
    lista.append(df)

halfhourly_consumption = pd.concat(lista, axis=0, ignore_index=True)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


/Users/mariabelenalberti/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112


In [4]:
#Data prepatation
halfhourly_consumption.head(),halfhourly_consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167817021 entries, 0 to 167817020
Data columns (total 3 columns):
LCLid             object
tstp              object
energy(kWh/hh)    object
dtypes: object(3)
memory usage: 3.8+ GB


(       LCLid                         tstp energy(kWh/hh)
 0  MAC000027  2011-12-07 11:30:00.0000000         0.185 
 1  MAC000027  2011-12-07 12:00:00.0000000         0.155 
 2  MAC000027  2011-12-07 12:30:00.0000000         0.147 
 3  MAC000027  2011-12-07 13:00:00.0000000         0.164 
 4  MAC000027  2011-12-07 13:30:00.0000000         0.187 , None)

In [5]:
halfhourly_consumption['tstp']= pd.to_datetime(halfhourly_consumption['tstp'],format = '%Y-%m-%d %H:%M:%S')
halfhourly_consumption['year'] = pd.DatetimeIndex(halfhourly_consumption['tstp']).year

In [11]:
halfhourly_consumption.head()

,LCLid,tstp,energy(kWh/hh),year
0,MAC000027,2011-12-07 11:30:00,0.185,2011
1,MAC000027,2011-12-07 12:00:00,0.155,2011
2,MAC000027,2011-12-07 12:30:00,0.147,2011
3,MAC000027,2011-12-07 13:00:00,0.164,2011
4,MAC000027,2011-12-07 13:30:00,0.187,2011


In [6]:
# Filter for 2012 and 2013 (the only two complete years. We want them in order to take into account stationary data)
half_h = halfhourly_consumption[(halfhourly_consumption['year'] == 2012) | (halfhourly_consumption['year'] == 2013)]

In [7]:
LCLid_test = half_h.LCLid
LCLid_df_test = LCLid_test.to_frame()
LCLid_freq_test = LCLid_df_test['LCLid'].value_counts().reset_index()

In [8]:
LCLid_freq_test.head()

,index,LCLid
0,MAC004494,35089
1,MAC000027,35089
2,MAC000049,35089
3,MAC000048,35089
4,MAC000131,35089


In [12]:
# After looking into the data, that extra row was a null with odd tstp (not in the 30 min interval), hence we drop them
df_no_null = half_h[half_h['energy(kWh/hh)'] != "Null"]
LCLid = df_no_null.LCLid
LCLid_df = LCLid.to_frame()
LCLid_freq = LCLid_df['LCLid'].value_counts().reset_index()
LCLid_freq.head()

,index,LCLid
0,MAC004491,35088
1,MAC000027,35088
2,MAC004463,35088
3,MAC004494,35088
4,MAC004509,35088


In [13]:
#We save the LCLid in order to use it in the for loop
households_save = LCLid_freq.loc[LCLid_freq.LCLid >= 35000,:]
LCLid_complete = households_save['index'].values.tolist()
LCLid_complete.sort()

In [15]:
len(LCLid_complete)

348

In [23]:
# tstp complete for a merge
household_name_35088 = LCLid_freq.loc[LCLid_freq.LCLid == 35088,:]
household_name_35088.head()
household_35088 = df_no_null[df_no_null['LCLid'] == 'MAC004491']

In [27]:
tstp_complete = household_35088[['tstp']]

In [39]:
#Loop to create a dataframe for each household
i = 0
for household in LCLid_complete:
    household_mac = df_no_null[df_no_null['LCLid'] == LCLid_complete[i]]
    household_mac_cplt = pd.merge(tstp_complete,household_mac,on='tstp',how='outer')
    print(len(household_mac_cplt))
    del household_mac_cplt['LCLid']
    del household_mac_cplt['year']
    df = pd.DataFrame(household_mac_cplt)
    df.to_csv(str(household)+ ".csv",index=False)
    print(household + ' file nr ' +str(i))
    i = i + 1

35088
MAC000015 file nr 0
35088
MAC000017 file nr 1
35088
MAC000018 file nr 2
35088
MAC000019 file nr 3
35088
MAC000020 file nr 4
35088
MAC000021 file nr 5
35088
MAC000022 file nr 6
35088
MAC000023 file nr 7
35088
MAC000024 file nr 8
35088
MAC000025 file nr 9
35088
MAC000026 file nr 10
35088
MAC000027 file nr 11
35088
MAC000029 file nr 12
35088
MAC000030 file nr 13
35088
MAC000031 file nr 14
35088
MAC000032 file nr 15
35088
MAC000033 file nr 16
35088
MAC000034 file nr 17
35088
MAC000035 file nr 18
35088
MAC000036 file nr 19
35088
MAC000037 file nr 20
35088
MAC000038 file nr 21
35088
MAC000039 file nr 22
35088
MAC000040 file nr 23
35088
MAC000041 file nr 24
35088
MAC000042 file nr 25
35088
MAC000043 file nr 26
35088
MAC000044 file nr 27
35088
MAC000045 file nr 28
35088
MAC000046 file nr 29
35088
MAC000047 file nr 30
35088
MAC000048 file nr 31
35088
MAC000049 file nr 32
35088
MAC000051 file nr 33
35088
MAC000052 file nr 34
35088
MAC000053 file nr 35
35088
MAC000054 file nr 36
35088
MAC00

35088
MAC004547 file nr 297
35088
MAC004549 file nr 298
35088
MAC004550 file nr 299
35088
MAC004551 file nr 300
35088
MAC004552 file nr 301
35088
MAC004554 file nr 302
35088
MAC004555 file nr 303
35088
MAC004556 file nr 304
35088
MAC004557 file nr 305
35088
MAC004559 file nr 306
35088
MAC004561 file nr 307
35088
MAC004562 file nr 308
35088
MAC004563 file nr 309
35088
MAC004564 file nr 310
35088
MAC004565 file nr 311
35088
MAC004566 file nr 312
35088
MAC004567 file nr 313
35088
MAC004569 file nr 314
35088
MAC004570 file nr 315
35088
MAC004571 file nr 316
35088
MAC004572 file nr 317
35088
MAC004573 file nr 318
35088
MAC004574 file nr 319
35088
MAC004575 file nr 320
35088
MAC004576 file nr 321
35088
MAC004577 file nr 322
35088
MAC004579 file nr 323
35088
MAC004581 file nr 324
35088
MAC004583 file nr 325
35088
MAC004584 file nr 326
35088
MAC004845 file nr 327
35088
MAC004847 file nr 328
35088
MAC004848 file nr 329
35088
MAC004849 file nr 330
35088
MAC004850 file nr 331
35088
MAC004852 file